In [ ]:
pip install sentencepiece transformers

In [ ]:
pip install pandas numpy

In [ ]:
pip install tqdm

In [ ]:
pip install torch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from ast import literal_eval
import pickle
import time
from torch.utils.data import DataLoader
import torch.optim as optim

from transformers import T5ForConditionalGeneration, T5Tokenizer

summarizer_model = T5ForConditionalGeneration.from_pretrained("t5-large")
tokenizer = T5Tokenizer.from_pretrained("t5-large")

summarizer_model.eval()
for param in summarizer_model.parameters():
    param.requires_grad = False


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
news_df=pd.read_csv("/teamspace/studios/this_studio/Datasets/translated_articles.csv")
news_df

,Article_ID,Article
0,a81b0a9903de41d44ee65592838912460f3773114e1576...,"One of Norway's most popular studies, paramedi..."
1,c09f07e9eea49dd8cdf056227c0a36c36110bb8544edd4...,I hope that we will soon be finished. I am tir...
2,6639e650a768ce7dce91ce7313a7ff371dadf09fce33eb...,Have you had to wait a long time to get the ca...
3,f26d3d5b0ced3ce636a4be7c8338ecd6fa614889fd369d...,Footballer Jim Solbakken delivered a bottom-so...
4,db45cc8651bfdeb8d07f531bdab2262ffb55ae4916f107...,(Vålerenga Storhamar 52) Veteran Martin Røyma...
...,...,...
436,af6dcad6eab7278c50ab4a1841d6deb74dcbf12101489a...,"Otherwise, recent figures show that Chinese in..."
437,416fadd418ab8b655f73b9d841c6262c24b1c679757e0b...,(Sverige-Slovenia 11) Emil Forsberg (30) and B...
438,8e173aa2044b2a9eac2716f32f43478b4cdcc3928dfa7e...,BT's architecture critics give their recommend...
439,1e1be58590843755b5cd3647989201e8afeb5f38dbf41d...,What is it? King Harald and Queen Sonja were o...


In [3]:
nid2body = dict(zip(news_df["Article_ID"], news_df["Article"]))

# Example check
print(f"Total items: {len(nid2body)}")
sample_nid = list(nid2body.keys())[0]
print(f"Example:\nNID: {sample_nid}\nBody: {nid2body[sample_nid][:300]}")

Total items: 441
Example:
NID: a81b0a9903de41d44ee65592838912460f3773114e1576d0a8b15e387bfdbdcf
Body: One of Norway's most popular studies, paramedics, recently came to UiS. But it's not just grades that apply. Thursday was the entrance exam with physical tests.  The dream job is ambulances and paramedics. That's what I want most", says Ida Reilstad Flesjå. Among the breathing persons on the trainin


In [4]:
sum_df=pd.read_csv("/teamspace/studios/this_studio/Datasets/translated_summaries.csv")
sum_df

,Summary_id,Summary,Source,Article_id
0,U_0_S_1,"Despite obstacles such as knee injuries, stude...","One of Norway's most popular studies, paramedi...",a81b0a9903de41d44ee65592838912460f3773114e1576...
1,U_0_S_2,Summary: Daniel Vigdel Hansen and Rakel Helghe...,I hope that we will soon be finished. I am tir...,c09f07e9eea49dd8cdf056227c0a36c36110bb8544edd4...
2,U_0_S_3,Never before have so many taken the driver's c...,Have you had to wait a long time to get the ca...,6639e650a768ce7dce91ce7313a7ff371dadf09fce33eb...
3,U_0_S_4,What is it? Footballer Jim Solbakken has seen ...,Football manager Jim Solbakken delivered a sol...,f26d3d5b0ced3ce636a4be7c8338ecd6fa614889fd369d...
4,U_0_S_5,What is it? In the final match between Våleren...,(Vålerenga Storhamar 52) Veteran Martin Røyma...,db45cc8651bfdeb8d07f531bdab2262ffb55ae4916f107...
...,...,...,...,...
1094,U_38_S_176,Solbak a veterinarian who works at Anicura ve...,But sometimes the hospital doesn't have blood ...,94d75ba68a891e27f00f04e16959c3dce449b4e46f63c2...
1095,U_38_S_177,Professor George Church told the BBC that they...,They will try to splice this DNA with genes fr...,17a29b85dc9bfa1cb2a2d04b324545a26843c707c3996c...
1096,U_38_S_178,The Statistical Institute confirms that the ac...,"Despite the decline, the activity in the commo...",e2184d047286618e2e90d0981aba083e042b665fc3f0aa...
1097,U_38_S_179,Reuters published that the Vatican owns more t...,"On Saturday, the Vatican published a list of i...",b820fc345a06cd4bbdad7a6922dae6eb1fef8619ee6e35...


In [5]:
# # Path to your sid2text pickle file
# with open('/teamspace/studios/this_studio/Data/sid2text.pkl', 'rb') as f:
#     sid2text = pickle.load(f)
# len(sid2text)

sid2text = dict(zip(sum_df["Summary_id"], sum_df["Summary"]))

# Example check
print(f"Total items: {len(sid2text)}")
sample_sid = list(sid2text.keys())[0]
print(f"Example:\nSID: {sample_sid}\nSummary: {sid2text[sample_sid][:300]}")

Total items: 1099
Example:
SID: U_0_S_1
Summary: Despite obstacles such as knee injuries, students like Thea Dyrli Djuve show that the way to qualify for the study can be successful for those who manage to pass the physical tests. Students such as Ida Reilstad Flesjå had to give up due to a knee injury, while Thea Dyrli Djuve managed to pass the t


In [6]:
import pickle
import torch
import pandas as pd

# === Device and Precision Setup ===
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hidden_dim = 768

# === Utility Functions ===
def KDE_PMI(emb1, emb2, bandwidth=0.7):
    bandwidth = torch.tensor(bandwidth, dtype=torch.float32, device=emb1.device)
    diff = emb1 - emb2
    sim = torch.exp(- (diff ** 2) / (2 * bandwidth ** 2))
    return sim

def get_embedding(key, table, dim):
    if key not in table:
        table[key] = torch.nn.Parameter(torch.randn(dim, dtype=torch.float32, device=device) * 0.01, requires_grad=True)
    return table[key]

# === Load Embeddings ===
def load_embeddings(path):
    with open(path, "rb") as f:
        data = pickle.load(f)
        if isinstance(data, dict):
            return {k: torch.tensor(v, dtype=torch.float32, device=device) for k, v in data.items()}
        elif isinstance(data, (list, tuple, np.ndarray)):
            # Create a dummy dict with index keys if needed
            return {str(i): torch.tensor(v, dtype=torch.float32, device=device) for i, v in enumerate(data)}
        else:
            raise ValueError(f"Unsupported embedding data format in {path}")

summary_embed = load_embeddings("/teamspace/studios/this_studio/summary_embeddings.pkl")
newsbody_embed = load_embeddings("/teamspace/studios/this_studio/newsbody_embeddings.pkl")
headline_embed = load_embeddings("/teamspace/studios/this_studio/headline_embeddings.pkl")

embed_tables = {
    'summary': summary_embed,
    'newsbody': newsbody_embed,
    'headline': headline_embed
}

# === Load Dataset ===
lookup_df = pd.read_csv("/teamspace/studios/this_studio/Datasets/Personalsum_Behavior_Vocab.csv").set_index('EdgeID')

# Use safe CSV loading for malformed file
train_df = pd.read_csv(
    "/teamspace/studios/this_studio/Datasets/personalsum_train_impersum.csv",
    on_bad_lines='skip',  # Skip rows with malformed quoting
    engine='python'       # More fault-tolerant parser
)



In [7]:
tail_set = set()

print("Building Tail2Idx from Bhist and Bpos...")
for row in tqdm(train_df.itertuples(), total=len(train_df), desc="Collecting tails"):
    try:
        bhist = literal_eval(row.Bhist)
        bpos = row.Bpos

        # Add tails from Bhist
        for b_id in bhist:
            if b_id in lookup_df.index:
                tail = lookup_df.loc[b_id, 'Tail']
                tail_set.add(tail)

        # Add tail from Bpos (in case it's not already in Bhist)
        if bpos in lookup_df.index:
            tail = lookup_df.loc[bpos, 'Tail']
            tail_set.add(tail)

    except Exception as e:
        print(f"[Skip] Error in row {row.Index}: {e}")

# === Final mappings ===
tail_ids = sorted(tail_set)
tail2idx = {tid: idx for idx, tid in enumerate(tail_ids)}
idx2tail = {idx: tid for tid, idx in tail2idx.items()}

print(f"Tail2Idx built: {len(tail2idx)} unique tail IDs.")

Building Tail2Idx from Bhist and Bpos...


Tail2Idx built: 1540 unique tail IDs.


In [8]:
class BehaviorEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.W_dwell = nn.Linear(1, hidden_dim, bias=False)
        self.gamma_rel = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_e = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.f = nn.Parameter(torch.empty(1, dtype=torch.float32, device=device))
        self.y_s = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_focus = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.y_acc = nn.Parameter(torch.empty(hidden_dim, dtype=torch.float32, device=device))
        self.K_short = nn.Parameter(torch.randn(30, dtype=torch.float32, device=device))  # last 50 steps
        self.K_long  = nn.Parameter(torch.randn(10, dtype=torch.float32, device=device))
        self.K_event = nn.Parameter(torch.randn(30, dtype=torch.float32, device=device))


        self.classifier = nn.Linear(hidden_dim, len(tail2idx))  # for tail_id classification
        self.bpos_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

        self.init_parameters()

    def init_parameters(self):
        nn.init.kaiming_uniform_(self.W_dwell.weight, nonlinearity='linear')
        for param in [self.gamma_rel, self.y_e, self.y_s, self.y_focus, self.y_acc]:
            nn.init.normal_(param, mean=0.0, std=0.01)
        with torch.no_grad():
            self.f.copy_(torch.tensor([0.5], dtype=torch.float32, device=device))

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        total_loss = torch.tensor(0., dtype=torch.float32, device=device)
        enc_loss = torch.tensor(0., dtype=torch.float32, device=device)
        pred_loss = torch.tensor(0., dtype=torch.float32, device=device)

        prev_e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)
        memory = []

        for t, b_id in enumerate(Bhist):
            if b_id not in lookup_df.index:
                continue

            row = lookup_df.loc[b_id]
            head_id, rel, tail_id = row['Head'], row['Relation'], row['Tail']
            dwell_s = row['dwell'] / 130000.0 if pd.notna(row['dwell']) else None

            head_emb = prev_e_b if t > 0 else embed_tables['headline'].get(head_id, torch.zeros(hidden_dim, device=device))
            e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)
            mem_e_b = torch.zeros(hidden_dim, dtype=torch.float32, device=device)
            

            if rel in ['click', 'skip', 'gen_summ']:
                tail_emb = embed_tables['newsbody'].get(tail_id, torch.zeros(hidden_dim, device=device))
                if rel == 'click':
                    dwell_v = self.W_dwell(torch.tensor([[dwell_s]], dtype=torch.float32, device=device)).squeeze(0) if dwell_s else torch.zeros(hidden_dim, device=device)
                    e_b = tail_emb + (dwell_v * self.y_e * KDE_PMI(tail_emb, head_emb))
                elif rel == 'skip':
                    e_b = self.f * tail_emb + (1. - self.f) * KDE_PMI(tail_emb, head_emb)
                elif rel == 'gen_summ':
                    hl_emb = embed_tables['headline'].get(tail_id, torch.zeros(hidden_dim, device=device))
                    e_b = self.y_s * KDE_PMI(tail_emb, head_emb) * hl_emb

            elif rel == 'summ_gen':
                sum_emb = embed_tables['summary'].get(tail_id, torch.zeros(hidden_dim, device=device))
                doc_emb = embed_tables['newsbody'].get(head_id, torch.zeros(hidden_dim, device=device))
                e_b = self.y_focus * KDE_PMI(sum_emb, head_emb) + self.y_acc * KDE_PMI(sum_emb, doc_emb)
            else:
                tail_emb = embed_tables['newsbody'].get(tail_id, torch.zeros(hidden_dim, device=device))
                e_b = tail_emb

            # Apply γ to e_b
            e_b = self.gamma_rel * e_b
            e_b_orig = e_b.clone()
            # --- MEMORY INJECTION ---
            if memory:
                idxs = torch.arange(len(memory)-1, -1, -1, device=device)
                memory_stack = torch.stack(memory)

                w_short = self.K_short[idxs.clamp(max=len(self.K_short)-1)]
                w_long  = self.K_long [idxs.clamp(max=len(self.K_long )-1)]
                w_event = self.K_event[idxs.clamp(max=len(self.K_event)-1)]

                m_short = (w_short.unsqueeze(1) * memory_stack).sum(0)
                m_long  = (w_long .unsqueeze(1) * memory_stack).sum(0)
                m_event = (w_event.unsqueeze(1) * memory_stack).sum(0)

                gates = torch.softmax(torch.stack([m_short, m_long, m_event]), dim=0)
                mem_e_b = gates[0]*m_short + gates[1]*m_long + gates[2]*m_event

            memory.append(mem_e_b.detach())  # Detach to stop gradient
            e_b = e_b_orig + mem_e_b
            prev_e_b = e_b

            # BCE or CE loss over current tail_id
            if tail_id in tail2idx:
                logits = self.classifier(e_b.unsqueeze(0))
                target = torch.tensor([tail2idx[tail_id]], dtype=torch.long, device=device)
                enc_loss += F.cross_entropy(logits, target)

        # === Final Step Prediction ===
        e_b_pos_pred = self.bpos_mlp(e_b)
        b_next = lookup_df.loc[Bpos]
        tpos_id = b_next['Tail']
        logits_pos = self.classifier(e_b_pos_pred.unsqueeze(0))
        target_pos = torch.tensor([tail2idx[tpos_id]], dtype=torch.long, device=device)
        pred_loss = F.cross_entropy(logits_pos, target_pos)

        total_loss = (0.5 * enc_loss + 0.5 * pred_loss)/(100*len(Bhist)+1)
        return e_b, e_b_pos_pred, total_loss


In [9]:

class SummaryInverseMapper(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, e_b_pos, e_b_hist, e_doc):
        x = torch.cat([e_b_pos, e_b_hist, e_doc], dim=-1)
        return self.fc(x)


class CrossAttentionAdapter(nn.Module):
    def __init__(self, query_dim, key_value_dim, hidden_dim):
        super().__init__()
        self.query_proj = nn.Linear(query_dim, hidden_dim)
        self.key_proj = nn.Linear(key_value_dim, hidden_dim)
        self.value_proj = nn.Linear(key_value_dim, hidden_dim)
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, query, key, value):
        if query.dim() == 1: query = query.unsqueeze(0)
        if key.dim() == 1: key = key.unsqueeze(0)
        if value.dim() == 1: value = value.unsqueeze(0)

        Q = self.query_proj(query).unsqueeze(1)  # [B, 1, H]
        K = self.key_proj(key).unsqueeze(1)      # [B, 1, H]
        V = self.value_proj(value).unsqueeze(1)  # [B, 1, H]

        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (Q.size(-1) ** 0.5)
        attn_weights = F.softmax(attn_scores, dim=-1)
        attended = torch.matmul(attn_weights, V)
        return self.out_proj(attended.squeeze(1))


class B2SModel(nn.Module):
    def __init__(self, encoder, inverse_mapper, adapter, summarizer_model, tokenizer, nid2body, sid2text):
        super().__init__()
        self.encoder = encoder
        self.inverse_mapper = inverse_mapper
        self.adapter = adapter
        self.summarizer = summarizer_model  # T5ForConditionalGeneration
        self.tokenizer = tokenizer
        self.nid2body = nid2body
        self.sid2text = sid2text
        self.adapter_to_t5 = nn.Linear(adapter.out_proj.out_features, summarizer_model.config.d_model)

    def forward(self, Bhist, Bpos, lookup_df, tail2idx, embed_tables):
        device = next(self.parameters()).device

        # Step 1: Encode user behavior
        e_b, e_b_pos_pred, loss_enc = self.encoder(Bhist, Bpos, lookup_df, tail2idx, embed_tables)

        # Step 2: Get document embedding and text
        head_id = lookup_df.loc[Bpos]['Head']
        tail_id = lookup_df.loc[Bpos]['Tail']
        e_doc = embed_tables['newsbody'].get(head_id, torch.zeros_like(e_b))
        doc_text = self.nid2body.get(head_id, "")

        # Step 3: Predict personalized summary embedding
        e_s_pos_pred = self.inverse_mapper(e_b_pos_pred, e_b, e_doc)
        adapted = self.adapter(e_s_pos_pred, e_doc, e_doc)
        adapted_proj = self.adapter_to_t5(adapted)  # [1, d_model]

        # Step 4: Encode document text with T5
        input_enc = self.tokenizer(doc_text, return_tensors="pt", padding=True, truncation=True).to(device)
        encoder_outputs = self.summarizer.encoder(
            input_ids=input_enc['input_ids'],
            attention_mask=input_enc['attention_mask'],
            return_dict=True
        )

        # Step 5: Prepare teacher-forced decoder inputs
        sid = lookup_df.loc[Bpos]['Tail']
        gold_summary = self.sid2text.get(sid, "")
        gold_inputs = self.tokenizer(gold_summary, return_tensors="pt", padding=True, truncation=True).to(device)
        decoder_input_ids = gold_inputs['input_ids'][:, :-1].contiguous()
        decoder_labels = gold_inputs['input_ids'][:, 1:].contiguous()

        # Step 6: Inject adapter vector as first token
        adapter_token_embed = adapted_proj.unsqueeze(1)  # [1, 1, d_model]
        dec_embed = self.summarizer.get_input_embeddings()(decoder_input_ids)  # [1, T, d_model]
        final_embed = torch.cat([adapter_token_embed, dec_embed], dim=1)

        decoder_labels = torch.cat([
            torch.full((1, 1), -100, dtype=torch.long, device=device),  # mask adapter loss
            decoder_labels
        ], dim=1)

        # Step 7: Decode with adapter-injected embeddings
        outputs = self.summarizer(
            encoder_outputs=encoder_outputs,
            decoder_inputs_embeds=final_embed,
            labels=decoder_labels
        )
        loss_txt = outputs.loss

        # Step 8: Decode predicted summary
        pred_ids = torch.argmax(outputs.logits, dim=-1)
        pred_summary = self.tokenizer.decode(pred_ids[0], skip_special_tokens=True)

        # Step 9: Return losses and summaries
        impersum_loss = 0.5 * loss_enc + 0.5 * loss_txt
        return (impersum_loss, loss_txt, loss_enc), pred_summary, gold_summary, doc_text


In [10]:
# === Define Supporting Modules ===
inverse_mapper = SummaryInverseMapper(hidden_dim).to(device)
adapter = CrossAttentionAdapter(query_dim=768, key_value_dim=768, hidden_dim=768).to(device)

encoder = BehaviorEncoder().to(device)

# === Initialize the Combined B2S Model ===
b2s_model = B2SModel(
    encoder=encoder,
    inverse_mapper=inverse_mapper,
    adapter=adapter,
    summarizer_model=summarizer_model,
    tokenizer=tokenizer,
    nid2body=nid2body,
    sid2text=sid2text
).to(device)


In [11]:
train_df

,Unnamed: 0,UserID,Bhist,Bpos
0,0,0_1,"['B1', 'B2', 'B3', 'B4']",B5
1,1,0_2,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']",B8
2,2,0_3,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8...",B12
3,3,0_4,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8...",B17
4,4,0_5,"['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8...",B20
...,...,...,...,...
1248,1248,38_234,"['B3965', 'B3966', 'B3967', 'B3968', 'B3969', ...",B4844
1249,1249,38_235,"['B3965', 'B3966', 'B3967', 'B3968', 'B3969', ...",B4847
1250,1250,38_236,"['B3965', 'B3966', 'B3967', 'B3968', 'B3969', ...",B4852
1251,1251,38_237,"['B3965', 'B3966', 'B3967', 'B3968', 'B3969', ...",B4855


In [12]:
train_rows = train_df.iloc[:300]
test_rows = train_df.iloc[300:1253]



def extract_pairs(df, lookup_df, tail2idx):
    data_pairs = []
    for _, row in df.iterrows():
        try:
            Bhist = literal_eval(row['Bhist'])
            Bpos = row['Bpos']  # explicitly given next behavior
            if Bpos in lookup_df.index:
                tail_id = lookup_df.loc[Bpos, 'Tail']
                if tail_id in tail2idx:
                    data_pairs.append((Bhist, Bpos))  # use full Bhist, and correct Bpos
        except:
            continue
    return data_pairs


train_data = extract_pairs(train_rows, lookup_df, tail2idx)
test_data = extract_pairs(test_rows, lookup_df, tail2idx)

print(f"Train samples: {len(train_data)} | Test samples: {len(test_data)}")

Train samples: 300 | Test samples: 953


In [13]:
# === Pick one sample ===
sample_idx = random.randint(0, len(train_data) - 1)
Bhist, Bpos = train_data[sample_idx]

print(f"\n=== Running B2S Forward Pass ===")
print(f"Sample Index: {sample_idx}")
print(f"Behavior History Length: {len(Bhist)}")
print(f"Bpos: {Bpos}")

b2s_model.eval()
with torch.no_grad():
    e_b, e_b_pos_pred, _ = b2s_model.encoder(Bhist, Bpos, lookup_df, tail2idx, embed_tables)

    head_id = lookup_df.loc[Bpos]['Head']
    e_doc = embed_tables['newsbody'].get(head_id, torch.zeros_like(e_b))

    e_s_pos_pred = b2s_model.inverse_mapper(e_b_pos_pred, e_b, e_doc)

    # Run model to get detailed losses
    (loss_total, loss_gen, loss_enc), pred_summary, gold_summary, generic_summary = b2s_model(
        Bhist, Bpos, lookup_df, tail2idx, embed_tables
    )

# === Metadata ===
true_tail_id = lookup_df.loc[Bpos]['Tail']
print(f"\nPredicted e_b_pos shape: {e_b_pos_pred.shape}")
print(f"True Tail ID (Bpos): {true_tail_id}")

# === Summaries ===
print("\n--- Personalized Summary Output ---")
print(f"Gold Summary   : {gold_summary}")
print(f"Predicted      : {pred_summary}")

# === Losses ===
print("\n--- Loss Breakdown ---")
print(f"Total Loss     : {loss_total:.4f}")
print(f"Gen Loss       : {loss_gen:.4f}")
print(f"Encoder Loss   : {loss_enc:.4f}")


=== Running B2S Forward Pass ===
Sample Index: 272
Behavior History Length: 26
Bpos: B1085


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.



Predicted e_b_pos shape: torch.Size([768])
True Tail ID (Bpos): U_13_S_3

--- Personalized Summary Output ---
Gold Summary   : Aalesund has struggled with weak results, with only one victory in the last nine matches. The match against Sarpsborg 08 was regarded as one of the worst performances, with players and coaches acknowledging lack of setup and quality. Despite changes and attempts at an excitement, the match ended with a superior victory for Sarpsborg 08, which has turned the trend with two subsequent victories after a long series of losses.
Predicted      : sden however   and and in  against Aps s  a as  of the   in with only and coaches ing  of  and.. , , an,,  match against with a superior and in Sarps und 08. which   the  in two wins winss in a.. of..

--- Loss Breakdown ---
Total Loss     : 2.0122
Gen Loss       : 3.9863
Encoder Loss   : 0.0381


In [14]:
import torch
from tqdm import tqdm
import os

# === Config ===
lr = 1e-3
num_epochs = 5
save_every = 1
checkpoint_template = "b2s_model_personalsum_epoch{:03d}.pt"
resume = True  # set to False if starting fresh
start_epoch = 0

# === Optimizer ===
optimizer = torch.optim.Adam(b2s_model.parameters(), lr=lr)

# === Load Checkpoint if Resume Enabled ===
if resume:
    checkpoint_files = [f for f in os.listdir() if f.startswith("b2s_model_openai_epoch") and f.endswith(".pt")]
    if checkpoint_files:
        last_ckpt = sorted(checkpoint_files)[-1]
        ckpt_epoch = int(last_ckpt.split("epoch")[-1].split(".pt")[0])
        start_epoch = ckpt_epoch
        b2s_model.load_state_dict(torch.load(last_ckpt))
        print(f"🔁 Loaded checkpoint '{last_ckpt}' and resumed from epoch {start_epoch + 1}")

# === Training Loop ===
for epoch in range(start_epoch, num_epochs):
    b2s_model.train()
    total_loss = 0.0
    total_gen_loss = 0.0

    print(f"\n🟦 Epoch {epoch + 1}/{num_epochs}")
    pbar = tqdm(train_data, desc=f"Training Epoch {epoch + 1}", dynamic_ncols=True)

    for Bhist, Bpos in pbar:
        optimizer.zero_grad()

        try:
            (loss_total, loss_txt, loss_enc), pred_summary, gold_summary, _ = b2s_model(
                Bhist, Bpos, lookup_df, tail2idx, embed_tables
            )
        except Exception as e:
            pbar.set_postfix_str(f"[Skip {Bpos}] {str(e)}")
            continue

        loss_total.backward()
        optimizer.step()

        total_loss += loss_total.item()
        total_gen_loss += loss_txt.item()

        pbar.set_postfix({
            "Loss": f"{loss_total.item():.4f}",
            "GenLoss": f"{loss_txt.item():.4f}"
        })

    avg_loss = total_loss / len(train_data)
    avg_gen_loss = total_gen_loss / len(train_data)
    print(f"✅ [Epoch {epoch + 1}] Avg Loss: {avg_loss:.4f} | Gen Loss: {avg_gen_loss:.4f}")

    if (epoch + 1) % save_every == 0:
        ckpt_path = checkpoint_template.format(epoch + 1)
        torch.save(b2s_model.state_dict(), ckpt_path)
        print(f"📌 Saved checkpoint: {ckpt_path}")



🟦 Epoch 1/5


Training Epoch 1: 100%|██████████| 300/300 [02:43<00:00,  1.83it/s, Loss=0.1192, GenLoss=0.2022]


✅ [Epoch 1] Avg Loss: 0.8503 | Gen Loss: 1.6631
📌 Saved checkpoint: b2s_model_personalsum_epoch001.pt

🟦 Epoch 2/5


Training Epoch 2: 100%|██████████| 300/300 [02:41<00:00,  1.85it/s, Loss=0.3458, GenLoss=0.6579] 


✅ [Epoch 2] Avg Loss: 0.2915 | Gen Loss: 0.5491
📌 Saved checkpoint: b2s_model_personalsum_epoch002.pt

🟦 Epoch 3/5


...

✅ [Epoch 3] Avg Loss: 0.2709 | Gen Loss: 0.5094
📌 Saved checkpoint: b2s_model_personalsum_epoch003.pt

🟦 Epoch 4/5


Training Epoch 4: 100%|██████████| 300/300 [02:39<00:00,  1.88it/s, Loss=0.2583, GenLoss=0.4852]


✅ [Epoch 4] Avg Loss: 0.3016 | Gen Loss: 0.5715
📌 Saved checkpoint: b2s_model_personalsum_epoch004.pt

🟦 Epoch 5/5


Training Epoch 5: 100%|██████████| 300/300 [02:37<00:00,  1.91it/s, Loss=0.2122, GenLoss=0.3933]


✅ [Epoch 5] Avg Loss: 0.2658 | Gen Loss: 0.5006
📌 Saved checkpoint: b2s_model_personalsum_epoch005.pt


In [15]:
# === Re-initialize your model architecture (same as training) ===
encoder = BehaviorEncoder().to(device)
inverse_mapper = SummaryInverseMapper(hidden_dim).to(device)
adapter = CrossAttentionAdapter(query_dim=768, key_value_dim=768, hidden_dim=768).to(device)

b2s_model = B2SModel(
    encoder=encoder,
    inverse_mapper=inverse_mapper,
    adapter=adapter,
    summarizer_model=summarizer_model,
    tokenizer=tokenizer,
    nid2body=nid2body,
    sid2text=sid2text
).to(device)

# === Load weights ===
checkpoint_path = "b2s_model_personalsum_epoch005.pt"  # <-- change as needed
b2s_model.load_state_dict(torch.load(checkpoint_path, map_location=device))
print(f"✅ Loaded model checkpoint: {checkpoint_path}")


✅ Loaded model checkpoint: b2s_model_personalsum_epoch005.pt


In [16]:
b2s_model

B2SModel(
  (encoder): BehaviorEncoder(
    (W_dwell): Linear(in_features=1, out_features=768, bias=False)
    (classifier): Linear(in_features=768, out_features=1540, bias=True)
    (bpos_mlp): Sequential(
      (0): Linear(in_features=768, out_features=768, bias=True)
      (1): ReLU()
      (2): Linear(in_features=768, out_features=768, bias=True)
    )
  )
  (inverse_mapper): SummaryInverseMapper(
    (fc): Sequential(
      (0): Linear(in_features=2304, out_features=768, bias=True)
      (1): ReLU()
      (2): Linear(in_features=768, out_features=768, bias=True)
    )
  )
  (adapter): CrossAttentionAdapter(
    (query_proj): Linear(in_features=768, out_features=768, bias=True)
    (key_proj): Linear(in_features=768, out_features=768, bias=True)
    (value_proj): Linear(in_features=768, out_features=768, bias=True)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
  )
  (summarizer): T5ForConditionalGeneration(
    (shared): Embedding(32128, 1024)
    (encoder): 

In [17]:
import pandas as pd
from tqdm import tqdm

b2s_model.eval()
results = []

with torch.no_grad():
    for Bhist, Bpos in tqdm(test_data, desc="🔍 Evaluating"):
        try:
            loss, pred_summary, gold_summary, _ = b2s_model(Bhist, Bpos, lookup_df, tail2idx, embed_tables)
            true_tail = lookup_df.loc[Bpos, 'Tail']
        except Exception as e:
            print(f"[Skip] {Bpos}: {e}")
            continue

        results.append({
            'Bpos': Bpos,
            'true_tail_id': true_tail,
            'generic summary':generic_summary,
            'pred_summary': pred_summary,
            'gold_summary': gold_summary
        })

df = pd.DataFrame(results)
df.to_csv("b2s_personalsum_eval_results.csv", index=False)

🔍 Evaluating: 100%|██████████| 953/953 [08:40<00:00,  1.83it/s]
